In [1]:
import os
import sys
import glob
import shutil
from copy import deepcopy

sys.path.append('..')
from preprocessing.ct_batch import CTImagesBatch
from dataset.dataset.dsindex import FilesIndex
print('OK')

OK


### Global index

In [2]:
DIR_TEST = '/notebooks/data/MRT/nci/*'
DIR_DUMP = '/notebooks/Koryagin/Batch_dev/dir_for_dumps/'

In [3]:
if os.path.exists(DIR_DUMP):
    shutil.rmtree(DIR_DUMP)

In [4]:
ind = FilesIndex(path=DIR_TEST, dirs=True)

### Create batch by subindexing

In [5]:
batch = CTImagesBatch(ind.create_subset(ind.index[[2,5,7]]))

In [6]:
batch.indices

array(['274a81c75d244187247789bd71de2b3a',
       '645e7f46eb9b834153ecf8e2b2921fe5',
       'd73264d2c4f71ef6bf8593ac55565d93'], 
      dtype='<U56')

### Test actions

* check load

In [7]:
batch.load(fmt='dicom')

[array([[[-1003,  -998,  -988, ..., -1014, -1013, -1007],
        [ -990,  -980,  -989, ...,  -996, -1010, -1017],
        [-1007,  -976,  -975, ...,  -991, -1017, -1023],
        ..., 
        [ -914,  -894,  -892, ...,   138,   159,   140],
        [ -912,  -899,  -908, ...,    97,    -2,  -131],
        [ -916,  -902,  -900, ...,  -249,  -381,  -518]],

       [[ -983,  -996, -1005, ..., -1017, -1018,  -998],
        [ -973,  -982, -1002, ..., -1013, -1019, -1004],
        [ -982,  -961,  -981, ..., -1011, -1019, -1005],
        ..., 
        [ -926,  -931,  -944, ...,   136,   164,   154],
        [ -934,  -942,  -946, ...,   119,    16,  -105],
        [ -928,  -923,  -911, ...,  -200,  -349,  -497]],

       [[-1015,  -998,  -980, ..., -1013, -1000,  -986],
        [-1015, -1004,  -986, ..., -1009, -1005,  -995],
        [-1017,  -998,  -981, ..., -1006, -1007,  -995],
        ..., 
        [ -906,  -917,  -917, ...,    94,   136,   152],
        [ -932,  -941,  -919, ...,   107,

In [8]:
batch_copy = deepcopy(batch)

* check attrs

In [9]:
batch._data.shape

(471, 512, 512)

In [10]:
batch._bounds

array([  0, 184, 314, 471])

In [11]:
batch.upper_bounds

array([184, 314, 471])

* check resize, segment, dump

In [12]:
batch2 = batch.resize(shape=(32, 64, 64), n_workers=6)

In [13]:
batch._data.shape

(96, 64, 64)

In [14]:
batch._bounds

array([ 0, 32, 64, 96])

* check dump

In [15]:
batch2.dump(dst=DIR_DUMP)

selector_events.py[LINE:53]#DEBUG    [2017-06-01 09:50:06,025]  Using selector: EpollSelector


[AttributeError("'CTImagesBatch' object has no attribute 'dump_blosc_async'",), AttributeError("'CTImagesBatch' object has no attribute 'dump_blosc_async'",), AttributeError("'CTImagesBatch' object has no attribute 'dump_blosc_async'",)]


ValueError: Failed while parallelizing

### Build batch from dump and compare

In [ ]:
ind_dumped = FilesIndex(path=DIR_DUMP + '/*', dirs=True)

In [ ]:
ind_dumped.index

In [ ]:
batch_from_dump = CTImagesBatch(ind_dumped)

In [ ]:
batch_from_dump.load(fmt='blosc', src=None)

In [ ]:
def plot_slices(height, num_pers, *batches):
    fig, axes = plt.subplots(1, len(batches), figsize = (10, len(batches)*8))
    com_args = dict(clim=(-1200, 300), cmap=(plt.cm.gray))
    
    for i in range(len(batches)):
        axes[i].imshow(batches[i].get_axial_slice(num_pers, height), **com_args)
    
    plt.show()

In [ ]:
batch.indices

In [ ]:
batch_from_dump.indices

In [ ]:
interact(lambda height: plot_slices(height, 1, batch_copy, batch, batch_from_dump), 
         height=(0.01, 0.99, 0.01))

### XIP

In [ ]:
xip = batch.make_xip()

In [ ]:
interact(lambda height: plot_slices(height, 2, batch_from_dump, xip), height=(0.01, 0.99, 0.01))

In [ ]:
xip.indices

In [ ]:
xip.index.index